In [ ]:
import sys; sys.path.append('..')
import numpy as np, elastic_rods
from bending_validation import suppress_stdout
from linkage_vis import LinkageViewer

l = elastic_rods.RodLinkage('../../examples/etienne_hyperboloid_measured.obj')
driver=l.centralJoint()

In [ ]:
crossSection = 0
openingSteps = 60
mat = elastic_rods.RodMaterial('+', 2000, 0.3, [10, 10, 1, 1])
l.setMaterial(mat)
with suppress_stdout(): elastic_rods.restlen_solve(l)
jdo = l.dofOffsetForJoint(driver)
fixedVars = list(range(jdo, jdo + 6)) # fix rigid motion for a single joint
with suppress_stdout(): elastic_rods.compute_equilibrium(l, fixedVars=fixedVars)

view = LinkageViewer(l, width=1024, height=768)
view.show()

In [ ]:
from open_linkage import open_linkage
def equilibriumSolver(tgtAngle, l, opts, fv):
    opts.beta = 1e-8
    opts.gradTol = 1e-8
    opts.niter = 200
    opts.useIdentityMetric = False
    opts.useNegativeCurvatureDirection = True
    return elastic_rods.compute_equilibrium(l, tgtAngle, options=opts, fixedVars=fv)

In [ ]:
with suppress_stdout(): open_linkage(l, driver, 1.0 * np.pi /16 - l.averageJointAngle, 3, view, zPerturbationEpsilon=0, equilibriumSolver=equilibriumSolver, maxNewtonIterationsIntermediate=30, verbose=True, useTargetAngleConstraint=True)

In [ ]:
mkdir indefinite_frames

In [ ]:
with suppress_stdout(): results = open_linkage(l, driver, 2 * np.pi / 16, openingSteps, view, zPerturbationEpsilon=0, equilibriumSolver=equilibriumSolver, maxNewtonIterationsIntermediate=30, verbose=True, useTargetAngleConstraint=True, outPathFormat='indefinite_frames/frame_{}.msh')

In [ ]:
cr = results[0][0]

In [ ]:
bendingEnergies = [cr.customData[-1]['energy_bend'] for cr in results[0]]
twistingEnergies = [cr.customData[-1]['energy_twist'] for cr in results[0]]
fullEnergy = [cr.energy[-1] for cr in results[0]]

In [ ]:
l.energy()

In [ ]:
from matplotlib import pyplot as plt
plt.plot(bendingEnergies)
plt.plot(twistingEnergies)
plt.plot(fullEnergy)

In [ ]:
mkdir indefinite_frames

In [ ]:
from matplotlib import pyplot as plt
import matplotlib

for i in range(1, 61):
    plt.figure(figsize=(9, 6))
    plt.plot(bendingEnergies[1:-1], linewidth=3)
    plt.plot(twistingEnergies[1:-1], linewidth=3)
    plt.plot(fullEnergy[1:-1], linewidth=3)
    plt.gca().add_artist(matplotlib.patches.Ellipse((i - 1, bendingEnergies[i]), 1.5, 2.8 * 1.5, color='C0'))
    plt.gca().add_artist(matplotlib.patches.Ellipse((i - 1, twistingEnergies[i]), 1.5, 2.8 * 1.5, color='C1'))
    plt.gca().add_artist(matplotlib.patches.Ellipse((i - 1, fullEnergy[i]), 1.5, 2.8 * 1.5, color='C2'))
    plt.tight_layout()
    plt.savefig(f'indefinite_frames/plot_{i - 1}.png')
    plt.close()

# In plane stretch and mode vibration

In [ ]:
import sys; sys.path.append('..')
import numpy as np, elastic_rods
from bending_validation import suppress_stdout
from linkage_vis import LinkageViewer

l = elastic_rods.RodLinkage('../../examples/etienne_hyperboloid_measured.obj')
driver=l.centralJoint()

In [ ]:
crossSection = 0
openingSteps = 60
mat = elastic_rods.RodMaterial('+', 2000, 0.3, [10, 10, 1, 1])
l.setMaterial(mat)
with suppress_stdout(): elastic_rods.restlen_solve(l)
jdo = l.dofOffsetForJoint(driver)
fixedVars = list(range(jdo, jdo + 6)) # fix rigid motion for a single joint
with suppress_stdout(): elastic_rods.compute_equilibrium(l, fixedVars=fixedVars)

view = LinkageViewer(l, width=1024, height=768)
view.show()

In [ ]:
from open_linkage import open_linkage
def equilibriumSolver(tgtAngle, l, opts, fv):
    opts.beta = 1e-8
    opts.gradTol = 1e-8
    opts.niter = 200
    opts.useIdentityMetric = False
    opts.useNegativeCurvatureDirection = True
    return elastic_rods.compute_equilibrium(l, tgtAngle, options=opts, fixedVars=fv)

In [ ]:
view.update()

In [ ]:
with suppress_stdout(): open_linkage(l, driver, 0.5 * np.pi /16 - l.averageJointAngle, 3, view, zPerturbationEpsilon=0, equilibriumSolver=equilibriumSolver, maxNewtonIterationsIntermediate=30, verbose=True, useTargetAngleConstraint=True)

In [ ]:
mkdir stretch_to_indefinite

In [ ]:
open_linkage(l, driver, 1.55 * np.pi /16 - l.averageJointAngle, 200, None, zPerturbationEpsilon=0, equilibriumSolver=equilibriumSolver, maxNewtonIterationsIntermediate=30, verbose=True, useTargetAngleConstraint=True, outPathFormat='stretch_to_indefinite/frame_{}.msh')

In [ ]:
H = l.hessian()

In [ ]:
import compute_vibrational_modes
lambdas, full_modes = compute_vibrational_modes.compute_vibrational_modes(l, fixedVars, compute_vibrational_modes.MassMatrixType.FULL)

In [ ]:
lambdas

In [ ]:
import mode_viewer
mview = mode_viewer.ModeViewer(l, full_modes, lambdas, amplitude=0.5)
mview.show()

In [ ]:
currDoFs = l.getDoFs()

In [ ]:
l.setDoFs(currDoFs)

In [ ]:
mkdir mode_frames

In [ ]:
modeVector = full_modes[:, 0]
paramVelocity = l.approxLinfVelocity(modeVector)
normalizedOffset = modeVector * (l.characteristicLength() / paramVelocity)
for i, t in enumerate(np.linspace(0, 2 * np.pi, 60)):
    l.setDoFs(currDoFs + 0.5 * np.sin(t) * normalizedOffset)
    l.saveVisualizationGeometry(f'mode_frames/frame_{i}.msh', averagedMaterialFrames=True)

In [ ]:
mview.setAmplitude(1)

In [ ]:
mkdir indefinite_frames

In [ ]:
with suppress_stdout(): results = open_linkage(l, driver, 2 * np.pi / 16, openingSteps, view, zPerturbationEpsilon=0, equilibriumSolver=equilibriumSolver, maxNewtonIterationsIntermediate=30, verbose=True, useTargetAngleConstraint=True, outPathFormat='indefinite_frames/frame_{}.msh')